In [ ]:
# hide
# dont_test
# default_exp http
%load_ext nb_black

<IPython.core.display.Javascript object>

# Async File Responses

In [ ]:
# export

import time


class AsyncResponseBase:
    headers = {"Content-Type": "application/octet-stream"}
    streaming = True
    _resource_closers = []
    is_async_fileresponse = True

    def __init__(self, *args, **kwargs):
        self.chunk_size = kwargs.get("chunk_size", 4096)
        self.status_code = kwargs.get("status_code", 200)
        self.raw_headers = kwargs.get("raw_headers", {})

    def get(self, header, alternate=None):
        return None

    async def send_stream_to_client(self, stream, send):
        started_serving = time.perf_counter()
        await send(
            {
                "type": "http.response.start",
                "status": self.status_code,
                "headers": self.raw_headers,
            }
        )
        chunks = []
        sent_size = 0

        more_body = True
        while more_body:
            chunk = await stream.read(self.chunk_size)
            chunk_len = len(chunk)
            more_body = chunk_len > 0
            chunks.append(chunk)
            await send(
                {
                    "type": "http.response.body",
                    "body": chunk,
                    "more_body": more_body,
                }
            )
            sent_size += chunk_len
        print("sent size: ", sent_size)
        self.elapsed = time.perf_counter() - started_serving

<IPython.core.display.Javascript object>

## Tests

In [ ]:
chunks = []

async def send(chunk):
    chunks.append(chunk)
    
    
class Stream:
    def __init__(self, data):
        self.index = 0
        self.data = data
        self.data_len = len(data)
    
    async def read(self, chunk_size):
        if self.index + chunk_size >= self.data_len:
            chunk = self.data[self.index:]
        else:
            chunk = self.data[self.index:self.index + chunk_size]
        self.index += chunk_size
        return chunk
            

content = "1234567890"
stream = Stream(content)
response = AsyncResponseBase(chunk_size=3)
await response.send_stream_to_client(stream, send)

sent_body = "".join([c["body"] for c in chunks if "body" in c])
assert content == sent_body

sent size:  10


## AiofileFileResponse

Serve a file asynchronously using [aiofiles](https://github.com/Tinche/aiofiles).

In [ ]:
# export

import aiofiles


class AiofileFileResponse(AsyncResponseBase):
    def __init__(self, path, chunk_size=4096):
        super().__init__(path, chunk_size=chunk_size)
        self.path = path

    def __repr__(self):
        return "<%(cls)s status_code=%(status_code)d>" % {
            "cls": self.__class__.__name__,
            "status_code": self.status_code,
        }

    async def stream(self, send):
        async with aiofiles.open(self.path, mode="rb") as stream:  # type: ignore
            await self.send_stream_to_client(stream, send)

<IPython.core.display.Javascript object>

### Tests

#### Complete FileResponse Testcase

* Create file with random content
* Stream file content via AsyncFileResponse
* Assert streamed content equals original content

In [ ]:
import os

from pathlib import Path

path = Path("testdata.bin")

with path.open("wb") as f:
    f.write(os.urandom(10000))

with path.open("rb") as f:
    content = f.read()

chunk_datas = []


async def send(chunk_data):
    chunk_datas.append(chunk_data)


response = AiofileFileResponse(path)
await response.stream(send)
received_content = b"".join([cd.get("body", b"") for cd in chunk_datas])

assert content == received_content

sent size:  10000


<IPython.core.display.Javascript object>

### test __repr__

In [ ]:
response = AiofileFileResponse("foobar")
assert "<AiofileFileResponse status_code=200>" == repr(response)

<IPython.core.display.Javascript object>

## AiobotocoreFileResponse

Serve a file asynchronously using [aiobotocore](https://github.com/aio-libs/aiobotocore).

In [ ]:
# export

import aiobotocore

from django.conf import settings


class AiobotocoreFileResponse(AsyncResponseBase):
    def __init__(self, bucket, key, chunk_size=4096):
        super().__init__(chunk_size=chunk_size)
        self.bucket = bucket
        self.key = key

    def __repr__(self):
        return "<%(cls)s status_code=%(status_code)d>" % {
            "cls": self.__class__.__name__,
            "status_code": self.status_code,
        }

    async def stream(self, send):
        session = aiobotocore.get_session()
        async with session.create_client(
            "s3",
            endpoint_url=settings.MINIO_ENDPOINT_URL,
            region_name=settings.DJANGO_AWS_REGION,
            aws_secret_access_key=settings.DJANGO_AWS_SECRET_ACCESS_KEY,
            aws_access_key_id=settings.DJANGO_AWS_ACCESS_KEY_ID,
            use_ssl=False,
        ) as client:
            minio_response = await client.get_object(Bucket=self.bucket, Key=self.key)
            async with minio_response["Body"] as stream:
                await self.send_stream_to_client(stream, send)

<IPython.core.display.Javascript object>

## Tests

In [ ]:
response = AiobotocoreFileResponse("bucket", "key")
assert "<AiobotocoreFileResponse status_code=200>" == repr(response)

<IPython.core.display.Javascript object>